In [1]:
import pygris
from pygris.data import get_census


In [2]:
STATE = "48" # Texas FIPS code

In [3]:
variables = {
    "S0801_C01_003E": "Drove Alone",
    "S0801_C01_046E": "Mean travel time",
    "S2001_C01_013E": "Median earnings",
}

In [ ]:
data = get_census(dataset = "acs/acs5/subject",
                           variables = list(variables.keys()),
                           year = 2022,
                           params = {
                             "for": "tract:*",
                             "in": "state:48"
                           },
                           guess_dtypes = True,
                           return_geoid = True
)

In [ ]:
data.rename(columns=variables, inplace=True)

In [ ]:
tracts = pygris.tracts(cb = True, state=STATE, year=2022)

In [ ]:
tracts = tracts.merge(data, on="GEOID", how="left")

In [ ]:
# Mean Daily drive time, all trip purposes
tracts["Component 1"] = (tracts["Mean travel time"] * (tracts["Drove Alone"]/100) * 2 * 3 )/60

In [ ]:
# Hours worked to afford a car spread over 7 days
tracts["Component 2"] = (12182/(tracts["Median earnings"]/40))/7

In [ ]:
tracts["Total Car Ownership Time"] = tracts["Component 1"] + tracts["Component 2"]

In [ ]:
tracts.explore("Total Car Ownership Time", tooltip=["Component 1", "Component 2", "Total Car Ownership Time"])

In [ ]:
tracts.to_file("tracts_time_wasted_on_cars.geojson")